- 均值和中位数填充缺失值操作回顾


In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series

In [7]:
df = DataFrame(data=np.random.randint(0,100,size=(6,4)))
df.iloc[1,3] = None
df.iloc[2,2] = np.nan
df.iloc[3,3] = None
df

,0,1,2,3
0,14,59,8.0,26.0
1,98,41,23.0,NaN
2,80,32,NaN,18.0
3,12,83,40.0,NaN
4,4,30,3.0,33.0
5,70,23,92.0,62.0


In [8]:
df1 = df.copy()

In [9]:
#使用均值对空值进行填充
for col in df.columns:
    if df[col].isnull().sum() > 0:
        df[col].fillna(value=np.mean(df[col]),inplace=True)

In [16]:
#使用中位数填充
for col in df1.columns:
    if df1[col].isnull().sum() > 0:
        df1[col].fillna(value=np.median(df1[col][df1[col].notnull()]),inplace=True)

In [17]:
df1

,0,1,2,3
0,14,59,8.0,26.0
1,98,41,23.0,29.5
2,80,32,23.0,18.0
3,12,83,40.0,29.5
4,4,30,3.0,33.0
5,70,23,92.0,62.0


### 级联操作
- pd.concat

pandas使用pd.concat函数，与np.   函数类似，只是多了一些参数：
```
objs
axis=0
keys
join='outer' / 'inner':表示的是级联的方式，outer会将所有的项进行级联（忽略匹配和不匹配），而inner只会将匹配的项级联到一起，不匹配的不级联
ignore_index=False
```

In [18]:
df1 = DataFrame(data=np.random.randint(0,100,size=(5,3)),columns=['a','b','c'])
df2 = DataFrame(data=np.random.randint(0,100,size=(5,3)),columns=['a','d','c'])

In [21]:
pd.concat((df1,df1),axis=1,join='outer')

,a,b,c,a,b,c
0,6,16,82,6,16,82
1,13,6,31,13,6,31
2,48,77,60,48,77,60
3,11,9,32,11,9,32
4,40,28,21,40,28,21


- 不匹配级联
    - 不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致
    - 有2种连接方式：
        - 外连接：补NaN（默认模式）
        - 内连接：只连接匹配的项

In [23]:
pd.concat((df1,df2),axis=0,join='inner')

,a,c
0,6,82
1,13,31
2,48,60
3,11,32
4,40,21
0,68,34
1,98,32
2,37,42
3,12,35
4,41,92


### 合并操作
- 级联是对表格做拼接，合并是对数据做合并
- merge与concat的区别在于，merge需要依据某一共同列来进行合并

- 使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

- 注意每一列元素的顺序不要求一致

#### 一对一合并

In [24]:
df1 = DataFrame({'employee':['Bob','Jake','Lisa'],
                'group':['Accounting','Engineering','Engineering'],
                })
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


In [25]:
df2 = DataFrame({'employee':['Lisa','Bob','Jake'],
                'hire_date':[2004,2008,2012],
                })
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012


In [26]:
pd.merge(left=df1,right=df2,how='inner',on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004


#### 一对多合并

In [27]:
df3 = DataFrame({
    'employee':['Lisa','Jake'],
    'group':['Accounting','Engineering'],
    'hire_date':[2004,2016]})
df3

,employee,group,hire_date
0,Lisa,Accounting,2004
1,Jake,Engineering,2016


In [28]:
df4 = DataFrame({'group':['Accounting','Engineering','Engineering'],
                       'supervisor':['Carly','Guido','Steve']
                })
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,Engineering,Steve


In [29]:
pd.merge(left=df3,right=df4,how='inner',on='group')

,employee,group,hire_date,supervisor
0,Lisa,Accounting,2004,Carly
1,Jake,Engineering,2016,Guido
2,Jake,Engineering,2016,Steve


#### 多对多合并

In [30]:
df1 = DataFrame({'employee':['Bob','Jake','Lisa'],
                 'group':['Accounting','Engineering','Engineering']})
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


In [31]:
df5 = DataFrame({'group':['Engineering','Engineering','HR'],
                'supervisor':['Carly','Guido','Steve']
                })
df5

,group,supervisor
0,Engineering,Carly
1,Engineering,Guido
2,HR,Steve


In [33]:
pd.merge(left=df1,right=df5,how='outer',on='group')

,employee,group,supervisor
0,Bob,Accounting,NaN
1,Jake,Engineering,Carly
2,Jake,Engineering,Guido
3,Lisa,Engineering,Carly
4,Lisa,Engineering,Guido
5,NaN,HR,Steve


In [35]:
pd.merge(left=df1,right=df5,how='right',on='group')

,employee,group,supervisor
0,Jake,Engineering,Carly
1,Lisa,Engineering,Carly
2,Jake,Engineering,Guido
3,Lisa,Engineering,Guido
4,NaN,HR,Steve


#### key的规范化


- 当两张表没有可进行连接的列时，可使用left_on和right_on手动指定merge中左右两边的哪一列列作为连接的列

In [36]:
df1 = DataFrame({'employee':['Bobs','Linda','Bill'],
                'group':['Accounting','Product','Marketing'],
               'hire_date':[1998,2017,2018]})
df1

,employee,group,hire_date
0,Bobs,Accounting,1998
1,Linda,Product,2017
2,Bill,Marketing,2018


In [37]:
df5 = DataFrame({'name':['Lisa','Bobs','Bill'],
                'hire_dates':[1998,2016,2007]})
df5

,name,hire_dates
0,Lisa,1998
1,Bobs,2016
2,Bill,2007


In [38]:
pd.merge(left=df1,right=df5,how='outer',left_on='employee',right_on='name')

,employee,group,hire_date,name,hire_dates
0,Bobs,Accounting,1998.0,Bobs,2016.0
1,Linda,Product,2017.0,NaN,NaN
2,Bill,Marketing,2018.0,Bill,2007.0
3,NaN,NaN,NaN,Lisa,1998.0


#### 内合并与外合并:out取并集 inner取交集

In [ ]:
df6 = DataFrame({'name':['Peter','Paul','Mary'],
               'food':['fish','beans','bread']}
               )
df7 = DataFrame({'name':['Mary','Joseph'],
                'drink':['wine','beer']})


In [ ]:
df6 = DataFrame({'name':['Peter','Paul','Mary'],
               'food':['fish','beans','bread']}
               )
df7 = DataFrame({'name':['Mary','Joseph'],
                'drink':['wine','beer']})
